# Systems and Solvers


`systems` are objects used in Underworld to numerically describe the following physical equations

* Steady State Heat Equation, a.k.a Poisson's equation.
* Stokes equation.
* Advection Diffusion equation.
* Advection equation.
* Thermomechanical Modelling

`systems` combine numerical algorithms with Underworld data structres, eg `MeshVariable`, `Swarm`, `Functions` and other input parameters; encapsulating the discretisation of the equation. A numerical `Solver` is then used to calculate a solution of a given `systems` object. 

**Keywords:** underworld systems, boundary conditions, heat equation, Stokes system, advection system

**References**

1. T. J. R. Hughes. The Finite Element Method: Linear Static and Dynamic Finite Element Analysis. Prentice-Hall, Inc. 1987

In [1]:
import underworld as uw

## Steady state heat equation

$
    \nabla(k\nabla)T + h = 0
$

where, 
* $T$ is the temperature variable.
* $k$ is a diffusivity term. 
* $h$ is a heating term.

This equation, given known $T$ values (the boundary conditions), solves for $T$ within the domain.

-------

In Underworld the `systems.SteadyStateHeat()` object is used to describe the equation above; its inputs are: 
* a `MeshVariable`, representing the temperature field and its boundary values
* 2 `Function`'s, representing $ k $ and $ h $. 
* the boundary conditions of $T$, defined with `DirichletCondition` or `NeumannCondition` objects.

`SteadyStateHeat` implements a galerkin finite element method to construct a system of linear equations. The linear system takes the following form,  

$ A x = b $,  

where $x$ is the unknown, in this case the temperature variable.

To solve the linear system for $x$, one creates a separate `Solver` object and calls its `solve()` method. This results in the `MeshVariable` (temperature) being updated with the FEM solution. 


For an example of the implementation of the Steady state heat equation see the notebook at `../test/SSHeatEq.ipynb`. 
  

--------
## Advection Diffusion Equations

In [2]:
help(uw.systems.AdvectionDiffusion)

Help on class AdvectionDiffusion in module underworld.systems._advectiondiffusion:

class AdvectionDiffusion(underworld._stgermain.StgCompoundComponent)
 |  This class provides functionality for a discrete representation
 |  of an advection-diffusion equation.
 |  
 |  The class uses the Streamline Upwind Petrov Galerkin SUPG method
 |  to integrate through time.
 |  
 |  .. math::
 |      \frac{\partial\phi}{\partial t}  + {\bf u } \cdot \nabla \phi= \nabla { ( k  \nabla \phi ) } + H
 |  
 |  
 |  Parameters
 |  ----------
 |  phiField : underworld.mesh.MeshVariable
 |      The concentration field, typically the temperature field
 |  phiDotField : underworld.mesh.MeshVariable
 |      A MeshVariable that defines the initial time derivative of the phiField.
 |      Typically 0 at the beginning of a model, e.g. phiDotField.data[:]=0
 |      When using a phiField loaded from disk one should also load the phiDotField to ensure
 |      the solving method has the time derivative information 

For an example of the implementation see
 `../examples/1_02_Convection_Example.ipynb`

--------------
## Stokes systems

$
    \nabla \cdot \left( 2 \eta \nabla \dot\varepsilon \right) - \nabla p = \mathbf{f}
$

$
    \nabla \cdot \mathbf{v} + Mp = h
$

where, 
* $\varepsilon_{ij}$ is a strain rate tensor, $\varepsilon_{ij} = 0.5 \left[ \mathbf{v}_{i,j} + \mathbf{v}_{j,i} \right]$, $\mathbf{v}$ is velocity.
* $p$ is a pressure-like variable.
* $\eta$ is viscosity.
* $ \mathbf{f} $ is a body force.
* $M$ and $h$ are compressibility parameters ( both set to 0 for incompressible flow)

The stokes system in Underworld is implemented with the `systems.Stokes()` object. It's inputs are:
* 2 `MeshVariable`, 1 for the velocity solution and 1 for the pressure like solution
* 2 `Function`'s, representing $ \eta $ and $ \mathbf{f} $. 
* the boundary conditions of $v$, defined using a `DirichletCondition`  or `NeumannCondition` object. 

The `Stokes` object implements a linear system of equations of the form:

$
\begin{bmatrix}
K & G \\ 
G^{T} & M
\end{bmatrix}\begin{bmatrix}u \\p \end{bmatrix}
= 
\begin{bmatrix}
F \\ 
h
\end{bmatrix}
$


To solve the linear system for $\begin{bmatrix}u \\p \end{bmatrix}$, one creates a separate `Solver` object and calls its `solve()` method. This results in the `MeshVariable`s (representing $u$ & $p$) being updated with the FEM solution.

See the user guide `/user_guide/09_StokesSolver.ipynb` for more details on solving the stokes system.

For an example of the implementation of the Stokes equation see   
`../examples/1_06_Rayleigh_Taylor.ipynb`


For an example with a stress condition see  
`../test/StokesEq_PoiseuilleFlow.ipynb`


-------
Particle advection
-----

Advection of a `Swarm` is handled by the `systems.SwarmAdvector` object. The `SwarmAdvector` is defined to act on a specific swarm with a given order; thereafter the user can control when it excutes to update the swarm's particle positions.

For an example of its use see, ``../examples/1_06_Rayleigh_Taylor.ipynb``.

-------
## Thermomechanical modelling

To simulate thermomechanical simulations Underworld weakly couples the stokes equations to the heat equation. This weak coupling, where solution variables are solved independently, is reflected in Underworld's implementation.
For an example of this see  
`examples/1_02_Convection_Example.ipynb`